In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import seal
from seal import *
import time
import pickle

In [2]:
#Data owner reloads encryption context

parms = EncryptionParameters(scheme_type.ckks)
parms.load('IDASH_parms')

context = SEALContext(parms)
ckks_encoder = CKKSEncoder(context)

In [6]:
keygen = KeyGenerator(context)
secret_key = keygen.secret_key()
secret_key.load(context, 'IDASH_secretkey')

public_key = keygen.create_public_key()
public_key.load(context, 'IDASH_pubkey')

scale = pickle.load(open('IDASH_scale','rb'))

In [7]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)

In [8]:
#Initialize and load ciphertexts from Model Owner, Step 3

results = []
for i in range(3):
    pt_init = ckks_encoder.encode(0.,scale)
    ct_init = encryptor.encrypt(pt_init)
    ct_init.load(context, 'IDASH_ct_results_%s' % i)
    results.append(ct_init)

In [9]:
decryptor = Decryptor(context, secret_key)

In [10]:
#Decrypt and decode results

final_scores = []
for i in range(3):
    pt_final = decryptor.decrypt(results[i])
    final_scores.append(ckks_encoder.decode(pt_final))

In [11]:
#DATA OWNER postprocessing of results
#The first 4 entries of scores are the correct entries of the matrix-vector product. 
final_junk_removed = []
for j in range(1000):
    final_junk_removed.append(final_scores[j // 341][24*(j%341):24*(j%341)+4])
    
final_junk_removed

[array([-0.0033247 ,  0.14498054,  0.15398586,  0.41065383]),
 array([0.16409535, 0.00957003, 0.00370445, 0.96284248]),
 array([ 0.47721409, -0.00475911, -0.00453028,  1.05160936]),
 array([ 1.06580134e-03,  1.00124973e+00, -9.31807669e-04,  2.19110844e-01]),
 array([0.00108172, 0.02776212, 1.06338454, 0.05005307]),
 array([ 0.00108656,  1.04114723, -0.00157359,  0.20561654]),
 array([ 0.7757917 , -0.00500792,  0.0038812 ,  1.04305448]),
 array([ 0.00108639,  1.04120326, -0.00297246,  0.22608339]),
 array([-0.00491046,  0.01484093,  0.97207738,  0.0277349 ]),
 array([ 9.65297614e-04,  9.93329904e-01, -2.14644872e-04,  2.09105944e-01]),
 array([-0.00401427,  0.14072373,  0.14213868,  0.4036849 ]),
 array([-0.00227118,  0.02394467,  1.06219754,  0.07307486]),
 array([-1.75579929e-04,  2.34223338e-02,  1.06342193e+00,  3.92600114e-02]),
 array([ 0.00112663,  1.00130967, -0.00180978,  0.24436932]),
 array([ 0.57506274, -0.00497245, -0.00437682,  1.06297747]),
 array([ 0.00112335,  1.002207

In [12]:
#DATA OWNER postprocessing
#Very simple function to convert final_junk_removed to list of label predictions
#Correct conversion assumes all scores in final_junk_removed round to 0 or 1
#Could update function to change scores >1.5 to 0 and <-.5 to 1
#This version miscategorizes all 0's as belonging to the 0th category
#and if more than one category has a 1, the reported category is the sum.
def conv_to_pred(score_array):
    predictions = []
    for scores in score_array:
        predictions.append(int(np.dot(np.round(scores),np.array([0,1,2,3]))))
        
    return predictions

In [13]:
from sklearn.metrics import classification_report,confusion_matrix

In [14]:
test_labels = pd.read_pickle('data_owner_labels')

In [15]:
pred = conv_to_pred(final_junk_removed)

In [16]:
#21 samples classified incorrectly, for 2% error.
#Likely all error coming from application of threshold polynomial.

print(confusion_matrix(pred,test_labels))

[[237   0   0   0   0]
 [  0 236   0   0   0]
 [  0   0 248   0   0]
 [ 19   0   0 258   0]
 [  0   0   2   0   0]]


In [17]:
#DATA OWNER postprocessing
#Very simple function to convert final_junk_removed to list of label predictions
#Just takes index of maximum score
def conv_to_pred2(score_array):
    predictions = []
    for scores in score_array:
        predictions.append(np.argmax(scores))
        
    return predictions

In [18]:
#This method fails for every sample in category 0
#but works for the other categories.
#Must figure out why.
#But for now use conv_to_pred

pred2 = conv_to_pred2(final_junk_removed)
print(confusion_matrix(pred2, test_labels))

[[  0   0   0   0]
 [  0 236   0   0]
 [  0   0 249   0]
 [256   0   1 258]]
